# Multi LLM LangGraph Agent with Function Calling, Context from Images, and Image Generation using Automatic1111
## Multi LLM LangGraph Agent

*Note: This lab runs 3 quantized LLMs locally for text generation.*  
*Additionally, it runs an SDXL stable diffusion model.*  
*You will need GPU, preferrably nVidia, with at least 20GB of VRAM, to run this lab.*  

*Note: about running on [Paperspace](https://www.paperspace.com/).*  
*This lab has been tested on Paperspace on their P6000 instance, which requires a Pro subscription.*  
*You will need to override the container image to `paperspace/gradient-base:pt211-tf215-cudatk120-py311-20240202` in order to run this on Paperspace since the default container comes with an older version of Python than what's needed for Automatic 1111 to run.*  

*For cells with shell commands, you should use the **Terminal** for best results.*  
*Start one terminal for each cell that has long running commands in the background, like running **Ollama**, **Automatic 1111**, **Streamlit**, and **local tunnel**.*  
*You can open more terminals for running other commands.*

### Disclaimer

This lab relies on the following:

1. [Ollama](https://ollama.ai) to run LLMs locally.
2. [Llama2](https://ollama.com/library/llama2) and [Bakllava](https://ollama.com/library/bakllava) LLMs hosted on Ollama. 
3. [a custom finetuned Mistral](https://ollama.com/klcoder/mistral-7b-functioncall) LLM for function calling hosted on Ollama.
4. [a function calling Prompt Template](https://smith.langchain.com/hub/klcoder/mistral-functioncalling) hosted on [Langchain Hub](https://smith.langchain.com/hub).

While all the code in this notebook, the function calling prompt template, as well as the custom finetuned LLMs are provided in good faith, run them at your own risk.

We recommend you exercise caution when running the code downloaded from the Internet, or from an untrusted source.
Always run such code in an isolated environment, like ephemeral containers, virtual machines, or cloud instances.

### Install prerequisite packages

In [18]:
%pip install -q langchain langchainhub langchain-core langchain-community langgraph pandas streamlit pydantic==2.*

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.1.9 requires pydantic!=1.8,!=1.8.1,<1.11.0,>=1.7.4, but you have pydantic 2.6.4 which is incompatible.
tensorflow 2.9.2 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.25.3 which is incompatible.
spacy 3.4.1 requires pydantic!=1.8,!=1.8.1,<1.10.0,>=1.7.4, but you have pydantic 2.6.4 which is incompatible.
confection 0.0.4 requires pydantic!=1.8,!=1.8.1,<1.11.0,>=1.7.4, but you have pydantic 2.6.4 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


### graph.py - code for LangGraph Image Generation Agent using Automatic1111

In [1]:
%%writefile graph.py
import base64
import json
import operator
import os
import requests
from langchain_community.chat_models import ChatOllama
from langchain_community.llms import Ollama
from langchain_core.messages import (
    BaseMessage,
    AIMessage,
    HumanMessage,
    FunctionMessage
)
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain.pydantic_v1 import BaseModel, Field
from langchain import hub
from langchain.tools import tool
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolInvocation, ToolExecutor
from numpy import random
from typing import Optional, TypedDict, Annotated, Sequence, Dict

# default base urls for automatic 1111 and ollama
a_1111_base_url = "http://localhost:7860"
ollama_base_url = "http://localhost:11434"

# environment variable names
a_1111_env_key = "AUTOMATIC1111_HOST_URL"
ollama_env_key = "OLLAMA_HOST_URL"

# override values for automatic 1111 and ollama from
# environment variables if present
if a_1111_env_key in os.environ:
    a_1111_base_url = os.environ[a_1111_env_key]

if ollama_env_key in os.environ:
    ollama_base_url = os.environ[ollama_env_key]

# bakllava model for image to text
image_llm = Ollama(model="bakllava",
                   base_url=ollama_base_url,
                   num_predict=100)

# llama2 model for text chat
text_llm = ChatOllama(model="llama2", base_url=ollama_base_url)

# mistral 7b functioncall model for function calling
fc_llm = ChatOllama(model="klcoder/mistral-7b-functioncall",
                    format="json", num_predict=100)


# config class for automatic 1111 image generation parameters with defaults
class Config(BaseModel):
    prompt: str
    negative_prompt: str = ''
    sampler_name: str = 'DPM++ 2M Karras'
    checkpoint_name: str = 'dreamshaperXL_v21TurboDPMSDE'
    batch_size: int = 1
    steps: int = 20
    seed: Optional[int] = None
    cfg_scale: int = 7
    width: int = 512
    height: int = 512
    denoising_strength: float = 0.7
    enable_hr: bool = False
    hr_scale: int = 2
    hr_upscaler: str = '4xUltrasharp_4xUltrasharpV10'
    hr_sampler_name: str = 'DPM++ 2M Karras'
    send_images: bool = True
    save_images: bool = True


# argument schema for txt2image tool
class Txt2ImageInput(BaseModel):
    prompt: str = Field(
        description="MidJourney style prompt for image generation")


# txt2image tool
@tool("txt2image", args_schema=Txt2ImageInput)
def txt2image(prompt: str, **kwargs) -> Dict:
    (
        "An image generation tool that takes in a prompt as string "
        "and returns a json response with images encoded in base64 string. "
        "The prompt is transformed from simple English "
        "to a comma separate MidJourney image generation prompt."
    )

    config = Config(prompt=prompt, **kwargs)
    if config.seed is None:
        config.seed = int(random.normal(scale=2**32))
    response = requests.post(a_1111_base_url + "/sdapi/v1/txt2img",
                             json=config.dict()).json()
    return response


# argument schema for image2text tool
class Image2TxtInput(BaseModel):
    prompt: str = Field(description="Question regarding the image")
    image: str = Field(description="Base64 encoded image")


# image2txt tool
@tool("image2txt", args_schema=Image2TxtInput)
def image2txt(prompt: str, image: str) -> str:
    (
        "An image description tool that takes "
        "in a question about an image or a picture as a prompt "
        "and returns the answer as string"
    )

    no_image_error = (
        "No image available within context. "
        "Upload an image or generate using prompt to describe it."
    )

    try:
        if image is None or len(image) == 0:
            return no_image_error
        _ = base64.b64decode(image)
    except Exception:
        return no_image_error

    bound = image_llm.bind(images=[image])
    response: str = bound.invoke(prompt)
    return response.strip()


# function to transform tool schema for function calling model
def tool_to_definition(tool):
    args = tool.args_schema.schema()
    args = {arg: args[arg] for arg in args if arg != 'title'}
    definition = {
        'name': tool.name,
        'description': tool.description.split(" - ")[-1],
        'parameters': args
    }
    return json.dumps(definition, indent=2)


tools = [image2txt, txt2image]
tool_descriptions = "\n\n".join([tool_to_definition(tool) for tool in tools])
tool_executor = ToolExecutor(tools)


# output format for function calling chain
class OutputFormat(BaseModel):
    function: Optional[str] = Field(description="Name of the function to call")
    arguments: Optional[Dict] = Field(
        description="Arguments or parameters to pass to the function")


# prompt template for function calling chain
fc_prompt = (
    hub
    .pull("klcoder/mistral-functioncalling")
    .partial(tools=tool_descriptions)
)

# function calling chain
fc_chain = fc_prompt | fc_llm | JsonOutputParser(pydantic_object=OutputFormat)

# prompt template for chat chain
text_prompt = PromptTemplate.from_template("""
You are a helpful agent. Respond to user questions honestly and truthfully.

Human: {question}
AI: """)

# text chat chain
text_chain = text_prompt | text_llm | StrOutputParser()


# state definition for langgraph agent
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    image: Optional[str]


# node function to execute tools
def call_tool(state):
    messages = state['messages']
    last_message = messages[-1]

    if isinstance(last_message, HumanMessage):
        return {"messages": []}

    tool_input = last_message.additional_kwargs

    if "image" in state and state["image"] is not None:
        tool_input["image"] = state["image"]

    action = ToolInvocation(
        tool=last_message.name,
        tool_input=tool_input,
    )

    response = tool_executor.invoke(action)
    function_message = FunctionMessage(
        content=json.dumps(response), name=action.tool)
    return {"messages": [function_message]}


# node function to call function calling model
def call_fc_model(state):
    messages = state["messages"]
    last_message = messages[-1]

    response = fc_chain.invoke({"question": last_message.content})
    if ('name' in response
            and response['name'] in [tool.name for tool in tools]):

        args = response["arguments"]

        if "image" in state and state["image"] is not None:
            args["image"] = state["image"]
        return {"messages": [AIMessage(
                    name=response["name"],
                    content="function",
                    additional_kwargs=args
                )]}
    else:
        return {"messages": [last_message]}


# node function for chat model
def call_model(state):
    messages = state["messages"]
    last_message = messages[-1]

    response = text_chain.invoke({"question": last_message.content})
    return {"messages": [AIMessage(content=response)]}


# conditional logic to determine which edge to take based on last message
def is_function_call(state):
    messages = state["messages"]
    last_message = messages[-1]

    if isinstance(last_message, HumanMessage):
        return "human"
    if (isinstance(last_message, AIMessage)
            and last_message.content == "function"):
        return "function"
    else:
        return "end"


# langgraph agent graph definition
workflow = StateGraph(AgentState)
workflow.add_node("functions", call_fc_model)
workflow.add_node("model", call_model)
workflow.add_node("tools", call_tool)
workflow.set_entry_point("functions")
workflow.add_conditional_edges(
    "functions",
    is_function_call,
    {
        "human": "model",
        "function": "tools",
        "end": END
    }
)
workflow.add_edge("tools", END)
workflow.add_edge("model", END)
app = workflow.compile()


Overwriting graph.py


### app.py - Code for Streamlit UI

In [2]:
%%writefile app.py
import base64
import json
import streamlit as st
from graph import app, ollama_base_url, a_1111_base_url
from langchain_core.messages import (
    AIMessage,
    HumanMessage,
    FunctionMessage,
    BaseMessage
)
from pandas.io.common import BytesIO
from PIL import Image

st.set_page_config(page_title="LangChain with Automatic 1111 API")
st.title("LangChain with Automatic 1111 API")

if "messages" not in st.session_state:
    st.session_state["messages"] = [AIMessage(content="How can I help you?")]

if "uploaded_file" not in st.session_state:
    st.session_state["uploaded_file"] = None

if "image" not in st.session_state:
    st.session_state["image"] = None

for msg in st.session_state.messages:
    if "image" not in msg.additional_kwargs:
        st.chat_message(msg.type).write(msg.content)
    else:
        st.chat_message(msg.type).image(
            msg.additional_kwargs["image"], width=512
        )
        if "params" in msg.additional_kwargs:
            with st.chat_message(msg.type).expander("Parameters"):
                st.code(msg.additional_kwargs["params"])

state = {}
with st.sidebar:
    st.text(f"Ollama\n{ollama_base_url}")
    st.text(f"Automatic 1111\n{a_1111_base_url}")

if uploaded_file := st.sidebar.file_uploader("Upload an image file",
                                             type=["jpg", "png"]):
    if st.session_state.uploaded_file != uploaded_file:
        st.session_state.uploaded_file = uploaded_file
        st.session_state.image = (
            base64
            .b64encode(uploaded_file.getvalue())
            .decode()
        )
        st.session_state.messages.append(
            HumanMessage(
                content=uploaded_file.name,
                additional_kwargs={
                    "image": uploaded_file,
                }
            )
        )
        st.chat_message("user").image(uploaded_file, width=512)

if prompt := st.chat_input():
    human_message = HumanMessage(content=prompt)
    state["messages"] = [human_message]
    st.session_state.messages.append(human_message)
    st.chat_message("human").write(prompt)

    response = ""
    if st.session_state.image is not None:
        image = st.session_state.image
        state["image"] = image

    response = app.invoke(state)
    messages = response["messages"]
    last_message = messages[-1]

    if isinstance(last_message, AIMessage):
        st.chat_message("assistant").write(last_message.content)
        st.session_state.messages.append(last_message)
    else:
        if isinstance(last_message, FunctionMessage):
            if last_message.name == "image2txt":
                last_message.content = str(last_message.content).strip('"')
                st.chat_message(last_message.name).write(last_message.content)
                st.session_state.messages.append(last_message)
            elif last_message.name == "txt2image":
                content = json.loads(str(last_message.content))
                imageb64 = content["images"][0]
                params = content["parameters"]
                params = json.dumps({p: params[p] for p in params if (
                    params[p] is not None and
                    params[p] != 0 and
                    params[p] is not False and
                    params[p] != "" and
                    params[p] != [] and
                    params[p] != {}
                )}, indent=2)
                image = Image.open(BytesIO(base64.b64decode(imageb64)))
                st.chat_message(last_message.name).image(image, width=512)
                with st.chat_message(last_message.name).expander("Parameters"):
                    st.code(params)
                st.session_state.messages.append(
                    BaseMessage(
                        name=last_message.name,
                        type=last_message.name,
                        content="",
                        additional_kwargs={
                            "params": params,
                            "image": image,
                        }
                    )
                )
                st.session_state.image = imageb64


Overwriting app.py


### Download and run ollama

We will use 3 local models for this exercise.

1. llama2 for casual conversation
2. bakllava for image to text conversation
3. mistral finetuned for function calling so we can correctly identify tools to use within the agent

To get started:

1. download the ollama binary
2. make it executable
3. start ollama in the background
4. download the hosted bakllava, llama2, and function calling model

### Download and start Ollama

In [3]:
%%bash --bg
curl -L https://ollama.ai/download/ollama-linux-amd64 -o ollama
chmod +x ollama
./ollama serve

### Download LLMs

In [4]:
%%bash
./ollama pull bakllava
./ollama pull llama2
./ollama pull klcoder/mistral-7b-functioncall

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest 
pulling deb26e54cceb...   0% ▕                ▏    0 B/4.1 GB                  pulling manifest 
pulling deb26e54cceb...   0% ▕                ▏    0 B/4.1 GB                  pulling manifest 
pulling deb26e54cceb...   0% ▕                ▏    0 B/4.1 GB                  pulling manifest 
pulling deb26e54cceb...   0% ▕                ▏    0 B/4.1 GB                  pulling manifest 
pulling deb26e54cceb...   0% ▕                ▏    0 B/4.1 GB                  pulling manifest 
pulling deb26e54cceb...   0% ▕                ▏    0 B/4.1 GB                  pulling manifest 
p

### Verify LLMs are loaded

In [5]:
%%bash
./ollama list

NAME                                  	ID          	SIZE  	MODIFIED               
klcoder/mistral-7b-functioncall:latest	b892fbfe479b	4.2 GB	Less than a second ago	
bakllava:latest                       	3dd68bd4447c	4.7 GB	About a minute ago    	
llama2:latest                         	78e26419b446	3.8 GB	54 seconds ago        	


### Automatic1111

For text to image generation, we will use Automatic1111, a stable diffusion web ui client, that also serves an API that we can call directly.  
For this exercise, we will download the [DreamShaper XL](https://civitai.com/models/112902/dreamshaper-xl) model and [4xUltrasharp](https://civitai.com/models/116225?modelVersionId=125843) rescaler. 

### Download Automatic1111 installer scripts

In [6]:
%%bash
curl -L https://raw.githubusercontent.com/AUTOMATIC1111/stable-diffusion-webui/master/webui.sh -o webui.sh
curl -L https://raw.githubusercontent.com/AUTOMATIC1111/stable-diffusion-webui/master/webui-user.sh -o webui-user.sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10163  100 10163    0     0   119k      0 --:--:-- --:--:-- --:--:--  119k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1332  100  1332    0     0  17076      0 --:--:-- --:--:-- --:--:-- 17076


### Download Dreamsharper model and 4xUltrasharp upscaler

In [7]:
%%bash
mkdir -p models/ESRGAN models/Stable-diffusion
curl -L https://civitai.com/api/download/models/351306 -o models/Stable-diffusion/dreamshaperXL_v21TurboDPMSDE.safetensors
curl -L https://civitai.com/api/download/models/125843 -o models/ESRGAN/4xUltrasharp_4xUltrasharpV10.pt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   519    0   519    0     0   2227      0 --:--:-- --:--:-- --:--:--  2227
100 6617M  100 6617M    0     0  48.5M      0  0:02:16  0:02:16 --:--:-- 45.9M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   494    0   494    0     0   2600      0 --:--:-- --:--:-- --:--:--  2586
100 63.8M  100 63.8M    0     0  58.1M      0  0:00:01  0:00:01 --:--:--  114M


### Install and start Automatic1111 with API mode enabled

In [11]:
%%bash --bg
chmod +x ./webui.sh
./webui.sh -f --api --listen --xformers --ckpt-dir=./models/Stable-diffusion --esrgan-models-path=/models/ESRGAN

### Start and background streamlit app

In [13]:
%%bash --bg
streamlit run app.py >/content/logs.txt

## Find the IP of your instance

In [14]:
!curl ipv4.icanhazip.com
!echo "Copy this IP into the webpage that opens below"

172.83.13.4
Copy this IP into the webpage that opens below


## Expose the Streamlit app on port 8501

In [16]:
!yes | npx localtunnel --port 8501

npm WARN exec The following package was not found and will be installed: localtunnel@2.0.2
m##################) ⠼ reify:yargs: http fetch GET 200 https://registry.npmjs.oistry.your url is: https://long-corners-check.loca.lt
^C
